<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [25]:
import pandas as pd
import psycopg2

In [26]:
##Введите параметры подключения 
#DBNAME =
#USER =
#PASSWORD =
#HOST =
#PORT =

In [27]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [244]:
query = f'''
            select
                count(v.id) vac_count
            from public.vacancies v
'''

In [245]:
df = pd.read_sql_query(query, connection)
df

,vac_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [246]:
query = f'''
            select
                count(e.id) employers_count
            from public.employers e
'''

In [247]:
df = pd.read_sql_query(query, connection)
df

,employers_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [248]:
query = f'''
            select
                count(a.id) areas_count
            from public.areas a
'''

In [249]:
df = pd.read_sql_query(query, connection)
df

,areas_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [250]:
query = f'''
            select
                count(i.id)
            from public.industries i
'''

In [251]:
df = pd.read_sql_query(query, connection)
df

,count,count
0,294,294


***

По предварительному анализу данных в таблицах содержится информация о 49197 вакансиях, представленных 23501 работадателями из 1362 регионов. Вакансии предложены по 294 сферам деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [254]:
query = f'''
            select
                a.name vac_in_area,
                count(v.id) cnt_vac
            from public.vacancies v
            left join public.areas a on v.area_id = a.id
            group by a.name
            order by cnt_vac desc
'''

In [255]:
df = pd.read_sql_query(query, connection)
df

,vac_in_area,cnt_vac
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [55]:
query = f'''
            select
                count(v.id)
            from public.vacancies v
            where (v.salary_from is not null) or (v.salary_to is not null)
'''

In [56]:
df = pd.read_sql_query(query, connection)
df

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [256]:
query = f'''
            select
                avg(v.salary_from) avg_sal_min,
                avg(v.salary_to) avg_sal_max
            from public.vacancies v
            
'''

In [257]:
df = round(pd.read_sql_query(query, connection))
df

,avg_sal_min,avg_sal_max
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [76]:
query = f'''
            select
                count(v.id),
                v.schedule,
                v.employment
            from public.vacancies v
            group by v.schedule, v.employment
            order by count(v.id) desc         
'''

In [77]:
df = pd.read_sql_query(query, connection)
df

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [78]:
query = f'''
            select
                count(v.id),
                v.experience
            from public.vacancies v
            group by v.experience
            order by count(v.id)
            
'''

In [79]:
df = pd.read_sql_query(query, connection)
df

,count,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

Наибольшее количество вакансий предлагают компании из Москвы и Санкт-Петербурга. Минимальный средний порог зарплаты по всем представленным регионам составляет 71 тыс., максимальный - 110 тыс (данные рассчитаны исходя из того, что меньше половины предприятий указали оба порога зарплат).
При этом работодателей интересуют сотрудники, готовые работать полный рабочий день, с небольшим опытом работы от 1 до 3 лет.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [280]:
query = f'''
            select
                count(v.id),
                e.name
            from public.vacancies v
            left join public.employers e on e.id = v.employer_id
            group by e.name
            order by count(v.id) desc
            limit 5

'''

In [281]:
df = pd.read_sql_query(query, connection)
df

,count,name
0,1,Торговый дом Аскания
1,2,Сенсум
2,1,Тойота Центр Омск / Лексус Омск
3,1,ФПГ Энергоконтракт
4,5,SoftTeco
...,...,...
14762,1,iAGE
14763,6,HR Prime
14764,2,Богатырь Комир
14765,1,AST Group Capital


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [258]:
query = f'''
            select
                a.name,
                count(distinct e.id) emp_count,
                count(v.id) vac_count
            from public.areas a
            left join public.employers e on e.area = a.id
            left join public.vacancies v on a.id = v.area_id
            where v.id is null
            group by a.id
            order by count(e.id) desc
            limit 1
'''

In [259]:
df = pd.read_sql_query(query, connection)
df

,name,emp_count,vac_count
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [272]:
query = f'''
            select
            e.name,
            count(distinct v.area_id) vac_in_area
            from public.employers e
            left join public.vacancies v on e.id = v.employer_id
            group by e.id
            order by count(v.area_id) desc
'''

In [273]:
df = pd.read_sql_query(query, connection)
df

,name,vac_in_area
0,Яндекс,181
1,Ростелеком,152
2,Тинькофф,43
3,СБЕР,24
4,Газпром нефть,24
...,...,...
23496,МАКМАХ,0
23497,Завод Стелкон,0
23498,ИнтерЭВМ,0
23499,Фёдоров Михаил,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [9]:
query = f'''
            select
            count(e.id)
            from public.employers e
            left join public.employers_industries ei on e.id = ei.employer_id
            where ei.industry_id is null
'''


In [10]:
df = pd.read_sql_query(query, connection)
df

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [17]:
query = f'''
            select
            e.name
            from public.employers e
            left join public.employers_industries ei on e.id = ei.employer_id
            group by e.name 
            having count(ei.industry_id) = 4
            order by e.name
            offset 2 limit 1
'''

In [18]:
df = pd.read_sql_query(query, connection)
df

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [30]:
query = f'''
            select
            count(ei.employer_id)
            from public.employers_industries ei
            left join public.industries i on i.id = ei.industry_id
            where i.name = 'Разработка программного обеспечения'
'''

In [31]:
df = pd.read_sql_query(query, connection)
df

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [275]:
import urllib.parse as req

path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) + req.quote(path_common.path) + '#' + req.quote(path_common.fragment)

df = pd.read_html(url)[1]
df['Город'] = df['Город'].replace('Воронеж[a]', 'Воронеж')

city_list = tuple(df['Город'])

In [154]:
query = f'''
            select
            a.name,
            count(v.id)
            
            from public.vacancies v
            left join public.employers e on v.employer_id = e.id
            left join public.areas a on a.id = v.area_id 
           
            where e.name = 'Яндекс' and (a.name like '{city_list[0]}' 
                            or a.name like '{city_list[1]}' or a.name like '{city_list[2]}' 
                            or a.name like '{city_list[3]}' or a.name like '{city_list[4]}'
                            or a.name like '{city_list[5]}' or a.name like '{city_list[6]}' 
                            or a.name like '{city_list[7]}' or a.name like '{city_list[8]}'        
                            or a.name like '{city_list[9]}' or a.name like '{city_list[10]}'
                            or a.name like '{city_list[11]}' or a.name like '{city_list[12]}' 
                            or a.name like '{city_list[13]}' or a.name like '{city_list[14]}'    
                            or a.name like '{city_list[15]}')       

            group by a.name      

            union

            select
                'total',
                sum(1)    
            from public.vacancies v
            left join public.employers e on v.employer_id = e.id
            left join public.areas a on a.id = v.area_id 
           
            where e.name = 'Яндекс' and (a.name like '{city_list[0]}' 
                            or a.name like '{city_list[1]}' or a.name like '{city_list[2]}' 
                            or a.name like '{city_list[3]}' or a.name like '{city_list[4]}'
                            or a.name like '{city_list[5]}' or a.name like '{city_list[6]}' 
                            or a.name like '{city_list[7]}' or a.name like '{city_list[8]}'        
                            or a.name like '{city_list[9]}' or a.name like '{city_list[10]}'
                            or a.name like '{city_list[11]}' or a.name like '{city_list[12]}' 
                            or a.name like '{city_list[13]}' or a.name like '{city_list[14]}'    
                            or a.name like '{city_list[15]}')    
'''

In [155]:
df = pd.read_sql_query(query, connection)
df

,name,count
0,total,485
1,Волгоград,24
2,Воронеж,32
3,Екатеринбург,39
4,Казань,25
5,Краснодар,30
6,Красноярск,23
7,Москва,54
8,Нижний Новгород,36
9,Новосибирск,35


***

Наибольшее количество вакансий среди работодателей предлагает Яндекс, им представлены вакансии в 181 регионе (из них 16 городов-миллионников, 485 вакансий (2% от общего количества вакансий)). 
Наименьшее количество вакансий (0) представлено в России, при доле работодателей 1.7% от общего их количества.
При этом 36% работодателей не указали сферу деятельности представленных вакансий. 15% представленных вакансий ориентированы на разработку программного обеспечения

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [262]:
query = f'''
            select
            count(v.id) ds_vac
            from public.vacancies v
            where v.name like '%data%' or v.name like '%DATA%' or v.name like '%Data%'
            or v.name like '%данн%' or v.name like '%ДАНН%' or v.name like '%Данн%'

'''

In [263]:
df = pd.read_sql_query(query, connection)
df

,ds_vac
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [264]:
# Список ключевых слов для поиска вакансий по дата-сайенс
v_ds_list = ['data scientist', 'data science', 'исследователь данных', 'ML', 'machine learning', 'машинн%обучен%']

query = f'''
            select
                count(v.id)
            from public.vacancies v
            
            --Выборка вакансий, относящихся к дата-сайенс
            where (lower(v.name) like '%{v_ds_list[0]}%' 
            or lower(v.name) like '%{v_ds_list[1]}%' or lower(v.name) like '%{v_ds_list[2]}%'
            or (v.name like '%{v_ds_list[3]}%' and v.name not like '%HTML%') or lower(v.name) like '%{v_ds_list[4]}%' or lower(v.name) like '%{v_ds_list[5]}%')
            
            -- Выборка по начинающим специалистам (junior)
            and 
            (lower(v.name) like '%junior%' or v.experience = 'Нет опыта' or v.employment = 'Стажировка')

'''

In [265]:
df = pd.read_sql_query(query, connection)
df

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [266]:
# Список ключевых слов для поиска вакансий по дата-сайенс
v_ds_list = ['data scientist', 'data science', 'исследователь данных', 'ML', 'machine learning', 'машинн%обучен%']

query = f'''
            select
                count(v.id)
            from public.vacancies v
            
            --Выборка вакансий, относящихся к дата-сайенс
            where (lower(v.name) like '%{v_ds_list[0]}%' 
            or lower(v.name) like '%{v_ds_list[1]}%' or lower(v.name) like '%{v_ds_list[2]}%'
            or (v.name like '%{v_ds_list[3]}%' and v.name not like '%HTML%') or lower(v.name) like '%{v_ds_list[4]}%' or lower(v.name) like '%{v_ds_list[5]}%')
            
            --Выборка вакансий c ключевыми навыками SQL/postgres
            and 
            (upper(v.key_skills) like '%SQL%' or lower(v.key_skills) like '%postgres%')

'''

In [267]:
df = pd.read_sql_query(query, connection)
df

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [216]:
# Список ключевых слов для поиска вакансий по дата-сайенс
v_ds_list = ['data scientist', 'data science', 'исследователь данных', 'ML', 'machine learning', 'машинн%обучен%']

query = f'''
            select
                count(v.id)
            from public.vacancies v
            
            --Выборка вакансий, относящихся к дата-сайенс
            where (lower(v.name) like '%{v_ds_list[0]}%' 
            or lower(v.name) like '%{v_ds_list[1]}%' or lower(v.name) like '%{v_ds_list[2]}%'
            or (v.name like '%{v_ds_list[3]}%' and v.name not like '%HTML%') or lower(v.name) like '%{v_ds_list[4]}%' or lower(v.name) like '%{v_ds_list[5]}%')
            
            --Выборка вакансий, в которых ключевым навыком является владение Python
            and 
            (v.key_skills like '%Python%')
            
'''

In [217]:
df = pd.read_sql_query(query, connection)
df

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [224]:
# Список ключевых слов для поиска вакансий по дата-сайенс
v_ds_list = ['data scientist', 'data science', 'исследователь данных', 'ML', 'machine learning', 'машинн%обучен%']

query = f'''
            select
                round(avg(1+length(v.key_skills)-length(Replace(v.key_skills, chr(9), ''))), 2)
            from public.vacancies v
            
            --Выборка вакансий, относящихся к дата-сайенс
            where (lower(v.name) like '%{v_ds_list[0]}%' 
            or lower(v.name) like '%{v_ds_list[1]}%' or lower(v.name) like '%{v_ds_list[2]}%'
            or (v.name like '%{v_ds_list[3]}%' and v.name not like '%HTML%') or lower(v.name) like '%{v_ds_list[4]}%' or lower(v.name) like '%{v_ds_list[5]}%')
            
            --Исключение вакансий, где не указаны ключевые навыки
            and v.key_skills is not null           
'''

In [225]:
df = pd.read_sql_query(query, connection)
df

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [268]:
# Список ключевых слов для поиска вакансий по дата-сайенс
v_ds_list = ['data scientist', 'data science', 'исследователь данных', 'ML', 'machine learning', 'машинн%обучен%']

query = f'''
            select
                round(avg(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)), 0)
            from public.vacancies v
            
            --Выборка вакансий, относящихся к дата-сайенс
            where (lower(v.name) like '%{v_ds_list[0]}%' 
            or lower(v.name) like '%{v_ds_list[1]}%' or lower(v.name) like '%{v_ds_list[2]}%'
            or (v.name like '%{v_ds_list[3]}%' and v.name not like '%HTML%') or lower(v.name) like '%{v_ds_list[4]}%' or lower(v.name) like '%{v_ds_list[5]}%')
            
            --Выборка вакансий, где указан или верхний или нижний порог зарплаты
            and (v.salary_from is not null or v.salary_to is not null)

            --Выборка вакансий по опыту работы
            and v.experience = 'От 3 до 6 лет'

            group by v.experience                     
'''

In [269]:
df = pd.read_sql_query(query, connection)
df

,round
0,243115.0


***

Порядка 3.5% всех вакансий относятся к работе с данными, из них: 2.8% подходят для начинающих дата-сайентистов; 11.4% вакансий подходят для специалистов с навыками в области SQL и postgres; 19.8% - для специалистов с навыками работы в Python. По вакансиям в области дата-сайенс специалист в среднем должен обладать 6 различными навыками. Средняя предлагаемая зарплата для специалиста в области дата-сайенс с опытом работы от 3 до 6 лет составляет поряддка 240 тыс.

# Общий вывод по проекту

In [312]:
query = f'''
            select
                e.name emp_name,
                avg(v.salary_from) sal_from,
                avg(v.salary_to) sal_to,
                a.name area_name,
                v.experience experience,
                ind.name ind_name

            from public.vacancies v
            left join public.employers e on e.id = v.employer_id
            left join public.areas a on v.area_id = a.id 
            left join public.employers_industries emp_ind on emp_ind.employer_id = v.employer_id
            left join public.industries ind on ind.id = emp_ind.industry_id
            
            --Выборка вакансий, относящихся к дата-сайенс
            where (lower(v.name) like '%{v_ds_list[0]}%' 
            or lower(v.name) like '%{v_ds_list[1]}%' or lower(v.name) like '%{v_ds_list[2]}%'
            or (v.name like '%{v_ds_list[3]}%' and v.name not like '%HTML%') or lower(v.name) like '%{v_ds_list[4]}%' or lower(v.name) like '%{v_ds_list[5]}%')
            
            --Выборка вакансий, где указан и верхний и нижний порог зарплаты
            and v.salary_from is not null and v.salary_to is not null

            group by e.name, v.experience, a.name, ind.name
            order by sal_to desc

            limit 30             
'''

In [313]:
df = pd.read_sql_query(query, connection)
df

,emp_name,sal_from,sal_to,area_name,experience,ind_name
0,Итон,200000.0,700000.0,Москва,От 3 до 6 лет,Разработка программного обеспечения
1,Итон,200000.0,700000.0,Москва,От 3 до 6 лет,"Системная интеграция, автоматизации технологи..."
2,AppQuantum,300000.0,400000.0,Москва,От 3 до 6 лет,"Маркетинговые, рекламные, BTL, дизайнерские, E..."
3,AppQuantum,300000.0,400000.0,Москва,От 3 до 6 лет,Разработка программного обеспечения
4,JSA Group,200000.0,400000.0,Москва,От 3 до 6 лет,"Системная интеграция, автоматизации технологи..."
5,AppQuantum,300000.0,400000.0,Москва,От 3 до 6 лет,"Интернет-компания (поисковики, платежные систе..."
6,JSA Group,200000.0,400000.0,Москва,От 3 до 6 лет,Разработка программного обеспечения
7,JSA Group,200000.0,400000.0,Санкт-Петербург,От 3 до 6 лет,Разработка программного обеспечения
8,JSA Group,200000.0,400000.0,Санкт-Петербург,От 3 до 6 лет,"Системная интеграция, автоматизации технологи..."
9,AppQuantum,300000.0,400000.0,Москва,От 3 до 6 лет,Издательская деятельность


Наибольший минимальный порог зарплаты в области дата-сайенс наблюдается в основном в Москве и Санкт-Петербурге, при этом для минимальной зарплаты 200-300тыс.р. опыт работы чаще всего должен составлять от 3 до 6 лет. Для соискателя с опытом работы до 3 трех лет минимальная ожидаемая зарплата по дата-сайенс ожидается в диапозоне 70-170 тыс.р. Основными сферами при таких заработной плате и опыте работы являются: разработка программного обеспечения и Системная интеграция, автоматизации технологий.
Максимальные зарплаты для тех же требований по опыту работы к соискателя и сфер деятельности составляют 350-700 тыс.р.

Общие выводы по проекту.

Наибольшее количесво вакансий представлено в крупнейших городах России, преимущественно в Москве и Санкт-Петербурге, при этом эти же города предлагают повышенные уровни минимальных и максимальных порогов зарплат, что связанно и с повышенным уровнем средней зарплаты в этих регионах. Основному анализу подлежали вакансии, подходящие дата-сайентистам, они предложены в основном в указанных выше городах. Наибольшая зарплата предлагается сотрудникам с опытом работы от 3 до 6 лет, однако и специалисти уровня junior в некоторых компаниях могут претендовать на зарплаты, которые предлагаются для специалистов с большим опытом работы. Яндекс - компания, представляющая наибольшее количество вакансий. Помимо большого опыта работы предприятия ищут сотрудников, готовых работать полный рабочий день, для соискателей удаленной работы предложено значительно меньшее количество вакансий. 
